In [80]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense,Dropout,MaxPooling1D, Flatten,BatchNormalization, GaussianNoise,Conv1D
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.utils import compute_class_weight
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, save_model, load_model
## some column names wrong, some csvs missing, 

In [81]:
#this was created in general/diagnosis_making notebook
diag = pd.read_csv("../general/ground_truth.csv").drop("Unnamed: 0", axis=1)

Below we are combining several clinical datasets.

In [82]:
demo = pd.read_csv("../PTDEMOG_07Nov2023.csv")

In [83]:
neuro = pd.read_csv("../NEUROEXM_07Nov2023.csv")

In [84]:
neuro.columns

Index(['Phase', 'ID', 'PTID', 'RID', 'SITEID', 'VISCODE', 'VISCODE2',
       'VISDATE', 'USERDATE', 'USERDATE2', 'EXAMDATE', 'NXVISUAL', 'NXAUDITO',
       'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER', 'NXHEEL',
       'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 'NXOTHER', 'NXABNORM',
       'update_stamp'],
      dtype='object')

In [85]:
clinical = pd.read_csv("../general/ADSP_PHC_COGN.csv").rename(columns={"PHASE":"Phase"})

In [86]:
clinical.head()

,RID,SUBJID,Phase,VISCODE,VISCODE2,EXAMDATE,PHC_VISIT,PHC_Sex,PHC_Education,PHC_Ethnicity,PHC_Race,PHC_AGE,PHC_Diagnosis,PHC_MEM,PHC_EXF,PHC_LAN,PHC_VSP,update_stamp
0,2,ADNI_011_S_0002,ADNI1,bl,bl,2005-09-08,1,1.0,16.0,2.0,5.0,74.439425,1.0,0.277,0.254,0.293,NaN,2022-10-18 09:15:45.0
1,2,ADNI_011_S_0002,ADNI1,m06,m06,2006-03-06,2,1.0,16.0,2.0,5.0,74.929500,1.0,0.237,0.278,0.535,-0.333,2022-10-18 09:15:45.0
2,2,ADNI_011_S_0002,ADNI1,m36,m36,2008-08-27,3,1.0,16.0,2.0,5.0,77.407255,1.0,0.344,0.419,0.535,NaN,2022-10-18 09:15:45.0
3,2,ADNI_011_S_0002,ADNIGO,m60,m60,2010-09-22,4,1.0,16.0,2.0,5.0,79.477070,1.0,0.101,0.066,0.620,0.264,2022-10-18 09:15:45.0
4,2,ADNI_011_S_0002,ADNI2,v06,m72,2011-09-19,5,1.0,16.0,2.0,5.0,80.468172,1.0,0.090,0.002,0.460,0.264,2022-10-18 09:15:45.0


In [87]:
diag["Subject"].value_counts()

068_S_2187    3
007_S_2394    3
067_S_2301    3
022_S_2379    3
031_S_2018    2
             ..
305_S_6899    1
020_S_6901    1
036_S_6897    1
137_S_6893    1
305_S_6810    1
Name: Subject, Length: 416, dtype: int64

In [88]:
comb = pd.read_csv("../general/DXSUM_PDXCONV_ADNIALL.csv")[["RID", "PTID" , "Phase"]]

In [89]:
m = comb.merge(demo, on = ["RID", "Phase"]).merge(neuro,on = ["RID", "Phase"]).merge(clinical,on = ["RID", "Phase"]).drop_duplicates()

In [90]:
m.columns = [c[:-2] if str(c).endswith(('_x','_y')) else c for c in m.columns]

m = m.loc[:,~m.columns.duplicated()]

In [91]:
diag = diag.rename(columns = {"Subject": "PTID"})

In [92]:
m = m.merge(diag, on = ["PTID", "Phase"])

In [93]:
m["PTID"].value_counts()

067_S_4767    18
141_S_0697    15
067_S_4072    14
067_S_4054    12
130_S_1200    10
              ..
016_S_4963     1
094_S_4459     1
016_S_4575     1
036_S_4740     1
131_S_6153     1
Name: PTID, Length: 109, dtype: int64

In [94]:
t = m

In [95]:
t.columns

Index(['RID', 'PTID', 'Phase', 'ID', 'SITEID', 'VISCODE', 'VISCODE2',
       'VISDATE', 'USERDATE', 'USERDATE2', 'PTSOURCE', 'PTGENDER', 'PTDOB',
       'PTDOBYY', 'PTHAND', 'PTMARRY', 'PTEDUCAT', 'PTWORKHS', 'PTNOTRT',
       'PTRTYR', 'PTRTYR_DRVD', 'PTHOME', 'PTTLANG', 'PTPLANG', 'PTCOGBEG',
       'PTMCIBEG', 'PTADBEG', 'PTADDX', 'PTETHCAT', 'PTRACCAT', 'update_stamp',
       'EXAMDATE', 'NXVISUAL', 'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE',
       'NXMOTOR', 'NXFINGER', 'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA',
       'NXGAIT', 'NXOTHER', 'NXABNORM', 'SUBJID', 'PHC_VISIT', 'PHC_Sex',
       'PHC_Education', 'PHC_Ethnicity', 'PHC_Race', 'PHC_AGE',
       'PHC_Diagnosis', 'PHC_MEM', 'PHC_EXF', 'PHC_LAN', 'PHC_VSP', 'Group'],
      dtype='object')

In [96]:
t = t.drop(["ID",  "SITEID", "VISCODE", "VISCODE2", "USERDATE", "USERDATE2",
            "update_stamp",  "PTSOURCE"], axis=1)  #"PTDOBMM", "DX"], axis=1) 
# these variables don't occur in the new DF. Ignorming them for now, but will
# investigate what they are

In [97]:
t.columns

Index(['RID', 'PTID', 'Phase', 'VISDATE', 'PTGENDER', 'PTDOB', 'PTDOBYY',
       'PTHAND', 'PTMARRY', 'PTEDUCAT', 'PTWORKHS', 'PTNOTRT', 'PTRTYR',
       'PTRTYR_DRVD', 'PTHOME', 'PTTLANG', 'PTPLANG', 'PTCOGBEG', 'PTMCIBEG',
       'PTADBEG', 'PTADDX', 'PTETHCAT', 'PTRACCAT', 'EXAMDATE', 'NXVISUAL',
       'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER',
       'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 'NXOTHER',
       'NXABNORM', 'SUBJID', 'PHC_VISIT', 'PHC_Sex', 'PHC_Education',
       'PHC_Ethnicity', 'PHC_Race', 'PHC_AGE', 'PHC_Diagnosis', 'PHC_MEM',
       'PHC_EXF', 'PHC_LAN', 'PHC_VSP', 'Group'],
      dtype='object')

In [98]:
t = t.fillna(-4)
t = t.replace("-4", -4)
cols_to_delete = t.columns[(t == -4).sum()/len(t) > .70]
t.drop(cols_to_delete, axis = 1, inplace = True)

# EVERYHTING UP TO HERE WORKS, THIS IS WHERE IT BEGINS TO BE PROBLEMATIC. I CAN'T
# KEEP WORKING NOW, BUT I WILL COME BACK TO THIS


# This drops PTWORKHS, which seems necessary for the next code block. I'm commenting
# that out for now because it doesn't seem all that relevant for analysis,
# but we should look at why we don't have PTWORK and if these deletions are necessary

In [99]:
len(t.columns)

47

In [100]:
# t["PTWORKHS"] = t["PTWORKHS"].str.lower().str.replace("housewife", "homemaker").str.replace("rn", "nurse").str.replace("bookeeper",
                                                                                                                #    "bookkeeper").str.replace("cpa", "accounting")
# I'm assuming PTWORKHS is what used to be PTWORK
# This part needs to be reviewed

In [102]:
# t["PTWORK"] = t["PTWORK"].fillna("-4").astype(str)

#again, dealing with the PTWORK is causing issues

In [ ]:
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*teach.*$)', 'education')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*bookkeep.*$)', 'bookkeeper')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*wife.*$)', 'homemaker')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*educat.*$)', 'education')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*engineer.*$)', 'engineer')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*eingineering.*$)', 'engineer') 
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*computer programmer.*$)', 'engineer') 
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*nurs.*$)', 'nurse')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*manage.*$)', 'managment')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*therapist.*$)', 'therapist')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*sales.*$)', 'sales')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*admin.*$)', 'admin')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*account.*$)', 'accounting')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*real.*$)', 'real estate')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*secretary.*$)', 'secretary')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*professor.*$)', 'professor')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*chem.*$)', 'chemist')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*business.*$)', 'business')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*writ.*$)', 'writing')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*psych.*$)', 'psychology')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*analys.*$)', 'analyst')

In [ ]:
cond = t['PTWORK'].value_counts()
threshold = 10
t['PTWORK'] = np.where(t['PTWORK'].isin(cond.index[cond >= threshold ]), t['PTWORK'], 'other')

In [ ]:
categorical = ['PTGENDER', 'PTWORK',
 'PTHOME',
 'PTMARRY',
 'PTEDUCAT',
 'PTPLANG',
 'NXVISUAL',
 'PTNOTRT',
 'NXTREMOR',
 'NXAUDITO',
 'PTHAND']

In [ ]:
quant = ['PTDOBYY',
 'PHC_MEM',
 'PHC_EXF',
 'PTRACCAT',
 'AGE',
 'PTADDX',
 'PTETHCAT',
 'PTCOGBEG',
 'PHC_VSP',
 'PHC_LAN']

In [ ]:
text = ["PTWORK", "CMMED"]

In [ ]:
cols_left = list(set(t.columns) - set(categorical) - set(text)  - set(["label", "Group","GROUP", "Phase", "RID", "PTID"]))
t[cols_left]

In [ ]:
for col in cols_left:
    if len(t[col].value_counts()) < 10:
        print(col)
        categorical.append(col)

In [ ]:
to_del = ["PTRTYR", "EXAMDATE", "SUBJECT_KEY", "PTWRECNT"]
t = t.drop(to_del, axis=1)

In [ ]:
quant = list(set(cols_left) - set(categorical) - set(text)  -set(to_del) - set(["label", "Group","GROUP", "Phase", "RID", "PTID"]))
t[quant]

In [ ]:
cols_left = list(set(cols_left) - set(categorical) - set(text) - set(quant) - set(to_del))

In [ ]:
#after reviewing the meaning of each column, these are the final ones
l = ['RID', 'PTID', 'Group', 'Phase', 'PTGENDER', 'PTDOBYY', 'PTHAND',
       'PTMARRY', 'PTEDUCAT', 'PTWORK', 'PTNOTRT', 'PTHOME', 'PTTLANG',
       'PTPLANG', 'PTCOGBEG', 'PTETHCAT', 'PTRACCAT', 'NXVISUAL',
       'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER',
       'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 
       'NXABNORM',  'PHC_MEM', 'PHC_EXF', 'PHC_LAN', 'PHC_VSP']

In [ ]:
t[l]

In [ ]:
dfs = []

In [ ]:
for col in categorical:
    dfs.append(pd.get_dummies(t[col], prefix = col))

In [ ]:
cat = pd.concat(dfs, axis=1)

In [ ]:
t[quant]

In [ ]:
cat

In [ ]:
t[["PTID","RID", "Phase", "Group"]]

In [ ]:
c = pd.concat([t[["PTID", "RID", "Phase", "Group"]].reset_index(), cat.reset_index(), t[quant].reset_index()], axis=1).drop("index", axis=1) #tex

In [ ]:
c

In [ ]:
#removing repeating subjects, taking the most recent diagnosis
c = c.groupby('PTID', 
                  group_keys=False).apply(lambda x: x.loc[x["Group"].astype(int).idxmax()]).drop("PTID", axis = 1).reset_index(inplace=False)

In [ ]:
c.to_csv("clinical.csv")

In [ ]:
#reading in the overlap test set
ts = pd.read_csv("overlap_test_set.csv").rename(columns={"subject": "PTID"})

#removing ids from the overlap test set
c = c[~c["PTID"].isin(list(ts["PTID"].values))]

In [ ]:
cols = list(set(c.columns) - set(["PTID","RID","subject", "ID","GROUP", "Group", "label", "Phase", "SITEID", "VISCODE", "VISCODE2", "USERDATE", "USERDATE2", "update_stamp", "DX_x","DX_y", "Unnamed: 0"]))
X = c[cols].values 
y = c["Group"].astype(int).values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
X_train.to_pickle("X_train_c.pkl")
y_train.to_pickle("y_train_c.pkl")

X_test.to_pickle("X_test_c.pkl")
y_test.to_pickle("y_test_c.pkl")